# Computer vision - Lab 8


## Agenda

- introduction to the camera and its parameters
- advanced image processing techniques,
- understanding the scene as schemes, objects and optical effects,
- advanced film processing techniques,



According to the [opencv documentation](https://github.com/opencv/opencv-python#installation-and-usage), only one package `opencv-python` (only main package) or `opencv-contrib-python` (main package + contrib + extra modules) should be installed and on colab there are two so reinstall them.

In [ ]:
!pip uninstall -y opencv-python
!pip uninstall -y opencv-contrib-python
!pip install opencv-contrib-python

Found existing installation: opencv-python 4.10.0.84
Uninstalling opencv-python-4.10.0.84:
  Successfully uninstalled opencv-python-4.10.0.84
Found existing installation: opencv-contrib-python 4.10.0.84
Uninstalling opencv-contrib-python-4.10.0.84:
  Successfully uninstalled opencv-contrib-python-4.10.0.84
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 9.8 MB/s eta 0:00:00


## Helpers

### Libraries


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import os
import sys
import glob
import random

from pprint import pprint
from ipywidgets import Video

from PIL import Image
from PIL.ExifTags import TAGS

### Datasets


* a frame from a football match between the Polish and England national teams and a fragment of the frame that will serve as a model for the pattern detection algorithm

In [ ]:
!gdown --id 1D-GbfGKqn982VG43SSB62-PIiv0QXuoZ
!gdown --id 17jZaa3iQO78Bjgk1W3EYBFE7RXKQ7gcM

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1D-GbfGKqn982VG43SSB62-PIiv0QXuoZ
To: /content/pl_eng.png
100% 853k/853k [00:00<00:00, 32.2MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=17jZaa3iQO78Bjgk1W3EYBFE7RXKQ7gcM
To: /content/template.png
100% 19.4k/19.4k [00:00<00:00, 35.3MB/s]


* excerpt from a scientific article on generating robot movements based on music of various types ([link](https://www.youtube.com/watch?v=kHBLaw5nfzk)) - the robot is located at the Poznań University of Technology,

In [ ]:
!gdown --id 1lJF5LX6m1KbgNGI2pJBQJVgEaU0VdUuF

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1lJF5LX6m1KbgNGI2pJBQJVgEaU0VdUuF
To: /content/anymal_cut.mp4
100% 948k/948k [00:00<00:00, 44.9MB/s]


* video from the repository OpenCV,

In [ ]:
!wget -O slow_traffic_small.mp4 https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4 -q --show-progress

slow_traffic_small. 100%[===================>]   1.92M  3.72MB/s    in 0.5s    


* clip of the English League match between Manchester United and Chelsea London.

In [ ]:
!gdown --id 16-H5SnEloiRgnSj_qtKNy0BTkJQZXi6c

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16-H5SnEloiRgnSj_qtKNy0BTkJQZXi6c
To: /content/free_kick.mp4
100% 510k/510k [00:00<00:00, 54.4MB/s]


### Visualization



In [ ]:
def imshow(a):
    a = a.clip(0, 255).astype("uint8")
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    display(PIL.Image.fromarray(a))

# Video

## Background removal

Due to the computational requirements, the class of algorithms used in video processing is based on iterative provision of information and its use.

One of the most important areas of work on video processing is detailing the background and moving elements.

Two iterative background masking algorithms are presented below ([A Duality Based Approach for Realtime TV-L1
Optical Flow](https://pequan.lip6.fr/~bereziat/cours/master/vision/papers/zach07.pdf)):
- cv2.createBackgroundSubtractorKNN()
- cv2.createBackgroundSubtractorMOG2()

These algorithms **learn** from the successive frames of the video, describing the background based on how the pixel and its surroundings have changed.

In [ ]:
anymal = cv2.VideoCapture("./anymal_cut.mp4")
if anymal.isOpened():
    print("Video loaded")

anymal_width = int(anymal.get(3))
anymal_height = int(anymal.get(4))

print(anymal_height, anymal_width)

anymal_fps = anymal.get(cv2.CAP_PROP_FPS)
print(anymal_fps)

Video loaded
360 640
30.0


In [ ]:
Video.from_file("anymal_cut.mp4")

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x0e1\x87mdat\x00\x…

In [ ]:
anymal_foreground_knn = cv2.VideoWriter(
    "anymal_cut_foreground_knn.avi",
    cv2.VideoWriter_fourcc(*"DIVX"),
    anymal_fps,
    (anymal_width, anymal_height),
    0,
)
anymal_foreground_mog2 = cv2.VideoWriter(
    "anymal_cut_foreground_mog2.avi",
    cv2.VideoWriter_fourcc(*"DIVX"),
    anymal_fps,
    (anymal_width, anymal_height),
    0,
)

foreground_knn = cv2.createBackgroundSubtractorKNN()
foreground_mog2 = cv2.createBackgroundSubtractorMOG2()

anymal.set(cv2.CAP_PROP_POS_FRAMES, 0)
while anymal.isOpened():
    ret, frame = anymal.read()

    if ret:
        anymal_foreground_knn.write(foreground_knn.apply(frame))
        anymal_foreground_mog2.write(foreground_mog2.apply(frame))
    else:
        break

anymal_foreground_knn.release()
anymal_foreground_mog2.release()

In [ ]:
!ffmpeg -hide_banner -loglevel error -i anymal_cut_foreground_knn.avi -y anymal_cut_foreground_knn.mp4
!ffmpeg -hide_banner -loglevel error -i anymal_cut_foreground_mog2.avi -y anymal_cut_foreground_mog2.mp4

In [ ]:
Video.from_file("anymal_cut_foreground_knn.mp4")

In [ ]:
Video.from_file("anymal_cut_foreground_mog2.mp4")

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x004R2mdat\x00\x00\x02…

## Object tracking

In addition to the separation of static and dynamic elements from the background, one can also distinguish a class of object tracking algorithms.

A big disadvantage of classic algorithms in this field is **the requirement to indicate the element that we want to track**. These algorithms only implement a tracking function, not a detection function.

The basic algorithms include:
- mean shift - an iterative algorithm that checks matches in the immediate vicinity of the current match and moves towards the best neighbor,
- cam shift - modification of the mean shift algorithm, which also adjusts the environment (window / bounding box) in subsequent steps,
- more advanced, also with the training phase:
  - Boosting,
  - MIL,
  - KCF,
  - TLF,
  - MedianFlow,
  - MOSSE,
  - CSRT

In [ ]:
traffic = cv2.VideoCapture("./slow_traffic_small.mp4")
if traffic.isOpened():
    print("Video loaded")

traffic_width = int(traffic.get(3))
traffic_height = int(traffic.get(4))

print(traffic_height, traffic_width)

traffic_fps = traffic.get(cv2.CAP_PROP_FPS)
print(traffic_fps)

Video loaded
360 640
29.97002997002997


In [ ]:
Video.from_file("./slow_traffic_small.mp4")

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x1eD\xe4mdat\x00\x…

In [ ]:
traffic.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, frame = traffic.read()
x, y, w, h = 300, 200, 100, 50  # simply hardcoded the values
track_window = (x, y, w, h)
roi = frame[y : y + h, x : x + w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(
    hsv_roi, np.array((0.0, 60.0, 32.0)), np.array((180.0, 255.0, 255.0))
)
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

In [ ]:
traffic_track = cv2.VideoWriter(
    "./slow_traffic_small_meanshift.avi",
    cv2.VideoWriter_fourcc(*"DIVX"),
    traffic_fps,
    (traffic_width, traffic_height),
)

traffic.set(cv2.CAP_PROP_POS_FRAMES, 0)
while traffic.isOpened():
    ret, frame = traffic.read()

    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        x, y, w, h = track_window
        traffic_track.write(cv2.rectangle(frame, (x, y), (x + w, y + h), 255, 2))
    else:
        break

traffic_track.release()

In [ ]:
!ffmpeg -hide_banner -loglevel error -i slow_traffic_small_meanshift.avi -y slow_traffic_small_meanshift.mp4

In [ ]:
Video.from_file("./slow_traffic_small_meanshift.mp4")

In [ ]:
traffic.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, frame = traffic.read()
x, y, w, h = 300, 200, 100, 50  # simply hardcoded the values
track_window = (x, y, w, h)
roi = frame[y : y + h, x : x + w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(
    hsv_roi, np.array((0.0, 60.0, 32.0)), np.array((180.0, 255.0, 255.0))
)
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

In [ ]:
traffic_track = cv2.VideoWriter(
    "./slow_traffic_small_camshift.avi",
    cv2.VideoWriter_fourcc(*"DIVX"),
    traffic_fps,
    (traffic_width, traffic_height),
)

traffic.set(cv2.CAP_PROP_POS_FRAMES, 0)
while traffic.isOpened():
    ret, frame = traffic.read()

    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = np.int0(cv2.boxPoints(ret))
        traffic_track.write(cv2.polylines(frame, [pts], True, 255, 2))
    else:
        break

traffic_track.release()

In [ ]:
!ffmpeg -hide_banner -loglevel error -i slow_traffic_small_camshift.avi -y slow_traffic_small_camshift.mp4

In [ ]:
Video.from_file("./slow_traffic_small_camshift.mp4")

In [ ]:
free_kick = cv2.VideoCapture("free_kick.mp4")
if free_kick.isOpened():
    print("Video loaded")

free_kick_width = int(free_kick.get(3))
free_kick_height = int(free_kick.get(4))

print(free_kick_height, free_kick_width)

free_kick_fps = free_kick.get(cv2.CAP_PROP_FPS)
print(free_kick_fps)

In [ ]:
def create_tracker(tracker_type):
    tracker_types = [
        "BOOSTING",
        "MIL",
        "KCF",
        "TLD",
        "MEDIANFLOW",
        "MOSSE",
        "CSRT",
    ]
    if tracker_type == 'BOOSTING':
        tracker = cv2.legacy.TrackerBoosting_create()
    if tracker_type == 'MIL':
        tracker = cv2.TrackerMIL_create()
    if tracker_type == 'KCF':
        tracker = cv2.TrackerKCF_create()
    if tracker_type == 'TLD':
        tracker = cv2.legacy.TrackerTLD_create()
    if tracker_type == 'MEDIANFLOW':
        tracker = cv2.legacy.TrackerMedianFlow_create()
    if tracker_type == 'MOSSE':
        tracker = cv2.legacy.TrackerMOSSE_create()
    if tracker_type == "CSRT":
        tracker = cv2.TrackerCSRT_create()

def draw_bbox(frame, bbox, color=(255, 255, 255)):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, color, 2, 1)

In [ ]:
free_kick.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, frame = free_kick.read()

player_bbox = (190, 260, 70, 100)
gkeeper_bbox = (375, 55, 60, 80)

draw_bbox(frame, player_bbox, (255, 0, 0))
draw_bbox(frame, gkeeper_bbox, (255, 255, 0))

imshow(frame)


In [ ]:
free_kick.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, frame = free_kick.read()

player_bbox = (190, 260, 70, 100)
gkeeper_bbox = (375, 55, 60, 80)

player_tracker = create_tracker("CSRT")
gkeeper_tracker = create_tracker("CSRT")

if player_tracker.init(frame, player_bbox):
    print("Player tracking algorithm initiated at point:", player_bbox)

if gkeeper_tracker.init(frame, gkeeper_bbox):
    print("Goalkeeper tracking algorithm initiated at point:", gkeeper_bbox)

free_kick_track = cv2.VideoWriter(
    "./free_kick_track.avi",
    cv2.VideoWriter_fourcc(*"DIVX"),
    free_kick_fps,
    (free_kick_width, free_kick_height),
)

free_kick.set(cv2.CAP_PROP_POS_FRAMES, 0)
while free_kick.isOpened():
    ret, frame = free_kick.read()

    if ret:
        ok, bbox = player_tracker.update(frame)
        if ok:
            draw_bbox(frame, bbox, (0, 255, 0))

        ok, bbox = gkeeper_tracker.update(frame)
        if ok:
            draw_bbox(frame, bbox, (0, 255, 0))

        free_kick_track.write(frame)
    else:
        break

free_kick_track.release()

In [ ]:
!ffmpeg -hide_banner -loglevel error -i free_kick_track.avi -y free_kick_track.mp4

In [ ]:
Video.from_file("./free_kick_track.mp4")

## Optical flow

Object tracking requires that you recognize and know some kind of object that you want to track. For example, by following the players on the pitch, we know what they look like (we have a pattern). The general case of object tracking is optical motion tracking.

For example, a soccer player can move forward. In the event that it is an attempt to kick the ball, the footballer will likely make a hand movement. Both the player (as an object) moved one way and the hands (as sub-objects) moved the other way.

Optical motion tracking goes even lower and analyzes the pixels and their surroundings, firstly identifying characteristic points in the image and secondly tracking them as separate objects.


In [ ]:
traffic = cv2.VideoCapture("./slow_traffic_small.mp4")
if traffic.isOpened():
    print("Video loaded!")

traffic_width = int(traffic.get(3))
traffic_height = int(traffic.get(4))

print(traffic_height, traffic_width)

traffic_fps = traffic.get(cv2.CAP_PROP_FPS)
print(traffic_fps)

In [ ]:
Video.from_file("./slow_traffic_small.mp4")

In [ ]:
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
)
# Create some random colors
color = np.random.randint(0, 255, (100, 3))

In [ ]:
traffic.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, old_frame = traffic.read()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
mask = np.zeros_like(old_frame)


In [ ]:
traffic_optical_flow = cv2.VideoWriter(
    "./slow_traffic_small_optical_flow.avi",
    cv2.VideoWriter_fourcc(*"DIVX"),
    traffic_fps,
    (traffic_width, traffic_height),
)

traffic.set(cv2.CAP_PROP_POS_FRAMES, 0)
while traffic.isOpened():
    ret, frame = traffic.read()

    if ret:
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        p1, st, err = cv2.calcOpticalFlowPyrLK(
            old_gray, frame_gray, p0, None, **lk_params
        )
        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(
                mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2
            )
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

        traffic_optical_flow.write(cv2.add(frame, mask))
    else:
        break

traffic_optical_flow.release()

In [ ]:
!ffmpeg -hide_banner -loglevel error -i slow_traffic_small_optical_flow.avi -y slow_traffic_small_optical_flow.mp4

In [ ]:
Video.from_file("./slow_traffic_small_optical_flow.mp4")